<a href="https://colab.research.google.com/github/masisparmo/capstone_project_gemini/blob/main/Capstone_project_pakeGeminiAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# ==============================================================================
# Coding Python ini dibuat dalam rangka menyelesaikan Capstone Project:
# HACKTIV8
# Student Development Initiative wave 3 (Juni 2025)
# Data Classification and Summarization Using IBM Granite (tapi ini versi yang pake GEMINI)
# [ONLINE] Data Batch 5 (instruktur: Manfred Michael)

# Oleh : ISPARMO
# ==============================================================================
# LANGKAH 1: INSTALASI DAN IMPORT LIBRARY
# Library ini seperti 'toolkit' atau 'perkakas' yang membantu kita
# untuk mengolah data, membuat grafik, dan memanggil AI.
# ==============================================================================
!pip install pandas matplotlib seaborn google-generativeai markdown

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import google.generativeai as genai
from google.colab import userdata
import markdown # Import library markdown

print("✅ Library berhasil diinstall dan diimport!")

# ==============================================================================
# LANGKAH 2: KONFIGURASI API GOOGLE (GEMINI)
# Mengambil kunci API yang sudah kita simpan di 'Secrets' tadi.
# ==============================================================================
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=google_api_key)
    print("✅ Kunci API Google (Gemini) berhasil dikonfigurasi.")
except Exception as e:
    print(f"🛑 Gagal mengambil kunci API Google. Pastikan Anda sudah menyimpannya di 'Secrets' dengan nama 'GOOGLE_API_KEY'. Error: {e}")

# ==============================================================================
# LANGKAH 3: MEMUAT DAN MEMAHAMI DATA BANJIR
# Kita 'baca' file CSV yang sudah diupload dan coba intip isinya.
# ==============================================================================
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Data_Banjir_Jakarta_2008-2025_BNBP_clean.csv'
try:
    # Membaca file CSV menjadi sebuah 'tabel data' yang disebut DataFrame
    df = pd.read_csv(file_path)

    # Mengubah kolom tanggal menjadi format tanggal yang dimengerti Python
    # errors='coerce' akan mengubah data yang formatnya salah menjadi kosong (NaT)
    # Menggunakan 'dt' sebagai nama kolom tanggal yang benar
    df['tanggal_kejadian'] = pd.to_datetime(df['dt'], errors='coerce')

    # Menghapus baris yang tanggalnya kosong setelah konversi
    df.dropna(subset=['tanggal_kejadian'], inplace=True)

    # Membuat kolom baru untuk tahun dan bulan agar mudah dianalisa
    df['tahun'] = df['tanggal_kejadian'].dt.year
    df['bulan'] = df['tanggal_kejadian'].dt.month_name()


    print("✅ Data berhasil dimuat dan diproses.")
    print("\nBerikut 5 baris pertama dari data Anda:")
    print(df.head())

    print("\nInformasi umum mengenai data:")
    df.info()

except FileNotFoundError:
    print(f"🛑 Error: File tidak ditemukan di path '{file_path}'. Pastikan Anda sudah mengupload file CSV ke Google Colab.")
except Exception as e:
    print(f"🛑 Terjadi error saat memuat data: {e}")


# ==============================================================================
# LANGKAH 4: ANALISA DAN VISUALISASI DATA
# Di sini kita akan membuat beberapa grafik untuk melihat tren banjir.
# ==============================================================================

# Atur style grafik agar terlihat lebih bagus
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# --- Analisa 1: Tren Jumlah Kejadian Banjir per Tahun ---
print("\nMembuat grafik tren banjir per tahun...")
kejadian_per_tahun = df['tahun'].value_counts().sort_index()

plt.figure() # Membuat kanvas baru
ax = sns.barplot(x=kejadian_per_tahun.index, y=kejadian_per_tahun.values, hue=kejadian_per_tahun.index, palette="viridis", legend=False)
ax.set_title('Jumlah Kejadian Banjir di Jakarta per Tahun (2008-2025)', fontsize=16)
ax.set_xlabel('Tahun', fontsize=12)
ax.set_ylabel('Jumlah Kejadian', fontsize=12)
plt.xticks(rotation=45)
# Menambahkan label angka di atas setiap bar
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')
plt.tight_layout()
plt.savefig('banjir_per_tahun.png') # Save plot as image
plt.close() # Close the plot figure
plt.show()

# --- Analisa 2: Bulan-Bulan Rawan Banjir ---
print("\nMembuat grafik bulan rawan banjir...")
# Tentukan urutan bulan yang benar
bulan_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
kejadian_per_bulan = df['bulan'].value_counts().reindex(bulan_order)

plt.figure()
ax = sns.barplot(x=kejadian_per_bulan.index, y=kejadian_per_bulan.values, hue=kejadian_per_bulan.index, palette="plasma", legend=False)
ax.set_title('Distribusi Kejadian Banjir Berdasarkan Bulan', fontsize=16)
ax.set_xlabel('Bulan', fontsize=12)
ax.set_ylabel('Jumlah Kejadian (akumulatif)', fontsize=12)
plt.xticks(rotation=45)
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')
plt.tight_layout()
plt.savefig('banjir_per_bulan.png') # Save plot as image
plt.close() # Close the plot figure
plt.show()

# --- Analisa 3: Wilayah Paling Sering Terdampak ---
print("\nMembuat grafik wilayah paling sering terdampak...")
# Menggunakan 'kabupaten' sebagai nama kolom wilayah yang benar
kejadian_per_wilayah = df['kabupaten'].value_counts()

plt.figure()
ax = sns.barplot(x=kejadian_per_wilayah.index, y=kejadian_per_wilayah.values, hue=kejadian_per_wilayah.index, palette="magma", legend=False)
ax.set_title('Wilayah Administrasi Jakarta yang Paling Sering Terdampak Banjir', fontsize=16)
ax.set_xlabel('Wilayah Administrasi', fontsize=12)
ax.set_ylabel('Jumlah Kejadian (akumulatif)', fontsize=12)
plt.xticks(rotation=45)
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 9), textcoords='offset points')
plt.tight_layout()
plt.savefig('banjir_per_wilayah.png') # Save plot as image
plt.close() # Close the plot figure
plt.show()

# Menyiapkan ringkasan data untuk diberikan kepada AI
summary_for_ai = f"""
Summary of Jakarta Flood Data (2008-2025):
1.  Flood incidents per year: {kejadian_per_tahun.to_dict()}
2.  Flood incidents per month (cumulative): {kejadian_per_bulan.to_dict()}
3.  Flood incidents per administrative region (cumulative): {kejadian_per_wilayah.to_dict()}
"""
print("\n--- Ringkasan Data untuk AI ---")
print(summary_for_ai)


# ==============================================================================
# LANGKAH 5: MENGGUNAKAN AI (LLM) UNTUK INSIGHT DAN REKOMENDASI
# Sekarang kita 'bertanya' kepada AI menggunakan data yang sudah kita olah.
# ==============================================================================
print("\nMemanggil AI untuk analisa dan rekomendasi... Ini mungkin butuh beberapa saat.")

# Membuat prompt (perintah) untuk AI dalam Bahasa Indonesia
prompt_untuk_ai = f"""
Anda adalah konsultan ahli perencanaan kota dan mitigasi bencana untuk Pemerintah Provinsi DKI Jakarta.
Berdasarkan ringkasan data historis banjir berikut, berikan analisis ringkas dan rekomendasi yang dapat ditindaklanjuti.

**Ringkasan Data:**
{summary_for_ai}

**Tugas Anda:**
1.  **Analisis Tren Utama:** Ringkaslah secara singkat tren paling penting yang diamati dari data. Tahun dan bulan manakah yang merupakan puncak kejadian banjir? Wilayah mana yang paling rentan?
2.  **Kemungkinan Akar Penyebab:** Berdasarkan tren ini, hipotesiskan kemungkinan penyebab yang mendasarinya (misalnya, musim hujan, masalah infrastruktur, penurunan permukaan tanah, pembangunan perkotaan).
3.  **Rekomendasi yang Dapat Ditindaklanjuti:** Berikan rekomendasi konkret berbasis data yang dikategorikan menjadi tiga tahapan:
    *   **Jangka Pendek (6 bulan ke depan):** Tindakan segera untuk persiapan menghadapi musim puncak berikutnya.
    *   **Jangka Menengah (1-3 tahun):** Perbaikan infrastruktur dan kebijakan.
    *   **Jangka Panjang (3+ tahun):** Inisiatif strategis berkelanjutan untuk masa depan kota.

Mohon susun respons Anda dengan jelas menggunakan judul untuk setiap bagian.
"""

# Memanggil model AI Gemini
try:
    # Pilih model Gemini yang sesuai (misalnya 'gemini-pro' atau 'gemini-2.5-flash')
    # Anda bisa melihat daftar model yang tersedia di dokumentasi Google AI Studio
    model = genai.GenerativeModel('gemini-2.5-flash')

    response = model.generate_content(prompt_untuk_ai)

    print("\n--- HASIL ANALISA DAN REKOMENDASI DARI AI (Markdown) ---")
    print(response.text)

    # Mengkonversi output Markdown ke HTML
    html_output = markdown.markdown(response.text)

    print("\n--- HASIL ANALISA DAN REKOMENDASI DARI AI (HTML) ---")
    # Menampilkan output HTML menggunakan display() agar dirender dengan benar di Colab
    from IPython.display import display, HTML
    display(HTML(html_output))

    print("\n✅ AI berhasil memberikan respon dan dikonversi ke HTML.")

except Exception as e:
    print(f"🛑 Gagal memanggil AI. Error: {e}")

✅ Library berhasil diinstall dan diimport!
✅ Kunci API Google (Gemini) berhasil dikonfigurasi.
✅ Data berhasil dimuat dan diproses.

Berikut 5 baris pertama dari data Anda:
                    dt                  kabupaten  jumlah_kejadian  meninggal  \
0  2025-05-23 09:15:00    Kota Adm. Jakarta Timur         1.000000          0   
1  2025-05-16 16:30:00    Kota Adm. Jakarta Timur         1.000000          0   
2  2025-05-15 08:10:00    Kota Adm. Jakarta Timur         1.000000          0   
3  2025-04-12 01:30:00    Kota Adm. Jakarta Timur         1.000000          0   
4  2025-03-18 10:21:00  Kota Adm. Jakarta Selatan         0.333333          0   

   hilang  luka_sakit  menderita  mengungsi  rumah_rusak_berat  \
0       0           0        351          0                  0   
1       0           0        450          0                  0   
2       0           0        390          0                  0   
3       0           0        125          0                  0   
4       0 


✅ AI berhasil memberikan respon dan dikonversi ke HTML.


In [37]:
# ==============================================================================
# LANGKAH 6: MEMBUAT LAPORAN HTML
# Menggabungkan grafik dan rekomendasi AI ke dalam satu file HTML.
# ==============================================================================

# Path untuk menyimpan file HTML
report_file_path = '/content/drive/MyDrive/Laporan_Analisa_Banjir_Jakarta.html'

# Membaca file gambar grafik sebagai data base64 agar bisa langsung di-embed di HTML
import base64
import os # Import os to check for file existence

def get_image_base64(image_path):
    try:
        if os.path.exists(image_path): # Check if file exists
            with open(image_path, "rb") as img_file:
                return base64.b64encode(img_file.read()).decode('utf-8')
        else:
            print(f"🛑 Error: File gambar tidak ditemukan di path '{image_path}'.")
            return ""
    except Exception as e:
        print(f"🛑 Terjadi error saat membaca file gambar '{image_path}': {e}")
        return ""


banjir_tahun_base64 = get_image_base64('banjir_per_tahun.png')
banjir_bulan_base64 = get_image_base64('banjir_per_bulan.png')
banjir_wilayah_base64 = get_image_base64('banjir_per_wilayah.png')

# Mendapatkan output HTML dari rekomendasi AI (sudah ada di variabel html_output)
# Pastikan variabel html_output sudah terisi dari eksekusi sebelumnya

# Membuat konten HTML
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Laporan Analisa Banjir Jakarta (2008-2025)</title>
    <style>
        body {{ font-family: Arial, sans-serif; line-height: 1.6; margin: 20px; }}
        h1, h2, h3 {{ color: #333; }}
        img {{ max-width: 100%; height: auto; display: block; margin: 20px auto; border: 1px solid #ddd; padding: 5px; }}
        .section {{ margin-bottom: 40px; padding: 20px; border: 1px solid #eee; border-radius: 8px; }}
        .ai-recommendations {{ background-color: #f9f9f9; }}
        pre {{ background-color: #eee; padding: 10px; overflow-x: auto; }}
        code {{ font-family: Consolas, Monaco, 'Andale Mono', 'Ubuntu Mono', monospace; }}
    </style>
</head>
<body>

    <h1>Laporan Analisa Banjir Jakarta (2008-2025)</h1>
    <p>Laporan ini menyajikan hasil analisa data historis kejadian banjir di Jakarta dari tahun 2008 hingga 2025, beserta rekomendasi dari model AI untuk mitigasi bencana.</p>

    <div class="section">
        <h2>Visualisasi Data</h2>
        <h3>Tren Jumlah Kejadian Banjir per Tahun</h3>
        <img src="data:image/png;base64,{banjir_tahun_base64}" alt="Grafik Tren Banjir per Tahun">

        <h3>Distribusi Kejadian Banjir Berdasarkan Bulan</h3>
        <img src="data:image/png;base64,{banjir_bulan_base64}" alt="Grafik Banjir per Bulan">

        <h3>Wilayah Administrasi Jakarta yang Paling Sering Terdampak Banjir</h3>
        <img src="data:image/png;base64,{banjir_wilayah_base64}" alt="Grafik Banjir per Wilayah">
    </div>

    <div class="section ai-recommendations">
        <h2>Analisa dan Rekomendasi dari AI</h2>
        {html_output}
    </div>

</body>
</html>
"""

print("✅ Struktur HTML laporan berhasil dibuat.")

# ==============================================================================
# LANGKAH 7: MENYIMPAN FILE HTML
# Menyimpan konten HTML ke dalam file di Google Drive.
# ==============================================================================
try:
    with open(report_file_path, "w", encoding="utf-8") as f:
        f.write(html_content)
    print(f"✅ Laporan HTML berhasil disimpan di: {report_file_path}")

except Exception as e:
    print(f"🛑 Gagal menyimpan laporan HTML. Error: {e}")

# ==============================================================================
# LANGKAH 8: INSTRUKSI UNTUK MEMBUKA FILE
# Memberikan petunjuk kepada pengguna cara membuka file yang disimpan.
# ==============================================================================
print("\n--- Cara Membuka Laporan ---")
print(f"Laporan HTML sudah tersimpan di Google Drive Anda pada path: {report_file_path}")
print("Untuk melihat laporan, silakan buka Google Drive, navigasi ke folder tersebut, dan buka file 'Laporan_Analisa_Banjir_Jakarta.html' menggunakan browser web.")
print("\n✅ Tugas pembuatan laporan HTML selesai. Silakan buka file tersebut dari Google Drive Anda.")

✅ Struktur HTML laporan berhasil dibuat.
✅ Laporan HTML berhasil disimpan di: /content/drive/MyDrive/Laporan_Analisa_Banjir_Jakarta.html

--- Cara Membuka Laporan ---
Laporan HTML sudah tersimpan di Google Drive Anda pada path: /content/drive/MyDrive/Laporan_Analisa_Banjir_Jakarta.html
Untuk melihat laporan, silakan buka Google Drive, navigasi ke folder tersebut, dan buka file 'Laporan_Analisa_Banjir_Jakarta.html' menggunakan browser web.

✅ Tugas pembuatan laporan HTML selesai. Silakan buka file tersebut dari Google Drive Anda.
